In [2]:
import pycaret
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# EDA

### Pasajeros diarios

In [3]:
# Dataset Pasajeros
url='https://raw.githubusercontent.com/jiang-011/AST_G4/main/dataset/Pasajeros_TP.csv'
pasajeros_d = pd.read_csv(url)
pasajeros_d.rename({'FechaRecaudacion': 'fecha', 'Cantidad_Pasajeros': 'pas'}, axis=1, inplace=True)
pasajeros_d = pasajeros_d[(pasajeros_d['fecha'] >= '2021-10-01') & (pasajeros_d['fecha'] <= '2023-10-30')]

# Para la fecha jueves 13 de octubre del 2022, se observa el valor de 1 y para corregir este dato se optó por sacar el promedio de los dos dias posteriores y el valor del día anterior.
# Además se agregan las observaciones que corresponden a los días 2022-05-17, 2022-10-12, 2023-06-13 y 2023-06-22 con el valor de 1, posteriormente se reemplaza este valor por el promedio de los dos dias posteriores y el valor del día anterior.
# Agregar fechas faltantes
nuevas_observaciones = pd.DataFrame([{'fecha': '2022-05-17', 'pas': 1},
                                     {'fecha': '2022-10-12', 'pas': 1},
                                     {'fecha': '2023-06-13', 'pas': 1},
                                     {'fecha': '2023-06-22', 'pas': 1}])
pasajeros_d = pd.concat([pasajeros_d, nuevas_observaciones], ignore_index=True)

# Ordenar el DataFrame por la columna de fechas
pasajeros_d = pasajeros_d.sort_values(by='fecha', ascending=True)

# Resetear el índice
pasajeros_d = pasajeros_d.reset_index(drop=True)

# Convertir la columna 'Fecha' a tipo datetime
pasajeros_d['fecha'] = pd.to_datetime(pasajeros_d['fecha'], dayfirst=True, format="%Y-%m-%d")

# Encontrar el índice correspondiente a '2022-05-17'
index_to_replace = pasajeros_d.index[pasajeros_d['fecha'] == '2022-05-17'][0]

# Calcular el promedio de una semana previa y una semana posterior
average_value_17 = (pasajeros_d.loc[index_to_replace - 7, 'pas'] + pasajeros_d.loc[index_to_replace + 7, 'pas']) / 2

# Redondear el promedio con 0 decimales
average_value_17 = round(average_value_17, 0)

# Reemplazar el valor en el DataFrame
pasajeros_d.loc[index_to_replace, 'pas'] = average_value_17

# Encontrar el índice correspondiente a '2022-10-12'
index_to_replace = pasajeros_d.index[pasajeros_d['fecha'] == '2022-10-12'][0]

# Calcular el promedio de un día previo y un día posterior
average_value_12 = (pasajeros_d.loc[index_to_replace - 7, 'pas'] + pasajeros_d.loc[index_to_replace + 7, 'pas']) / 2

# Redondear el promedio con 0 decimales
average_value_12 = round(average_value_12, 0)

# Reemplazar el valor en el DataFrame
pasajeros_d.loc[index_to_replace, 'pas'] = average_value_12

# Encontrar el índice correspondiente a '2022-10-13'
index_to_replace = pasajeros_d.index[pasajeros_d['fecha'] == '2022-10-13'][0]

# Calcular el promedio de un día previo y un día posterior
average_value_13 = (pasajeros_d.loc[index_to_replace - 7, 'pas'] + pasajeros_d.loc[index_to_replace + 7, 'pas']) / 2

# Redondear el promedio con 0 decimales
average_value_13 = round(average_value_13, 0)

# Reemplazar el valor en el DataFrame
pasajeros_d.loc[index_to_replace, 'pas'] = average_value_13

# Encontrar el índice correspondiente a '2023-06-13'
index_to_replace = pasajeros_d.index[pasajeros_d['fecha'] == '2023-06-13'][0]

# Calcular el promedio de un día previo y un día posterior
average_value_6 = (pasajeros_d.loc[index_to_replace - 7, 'pas'] + pasajeros_d.loc[index_to_replace + 7, 'pas']) / 2

# Redondear el promedio con 0 decimales
average_value_6 = round(average_value_6, 0)

# Reemplazar el valor en el DataFrame
pasajeros_d.loc[index_to_replace, 'pas'] = average_value_6

# Encontrar el índice correspondiente a '2023-06-22'
index_to_replace = pasajeros_d.index[pasajeros_d['fecha'] == '2023-06-22'][0]

# Calcular el promedio de un día previo y un día posterior
average_value_22 = (pasajeros_d.loc[index_to_replace - 7, 'pas'] + pasajeros_d.loc[index_to_replace + 7, 'pas']) / 2

# Redondear el promedio con 0 decimales
average_value_22 = round(average_value_22, 0)

# Reemplazar el valor en el DataFrame
pasajeros_d.loc[index_to_replace, 'pas'] = average_value_22

# Formato diario
pasajeros_d.set_index('fecha', inplace=True)

pasajeros_d = pasajeros_d.asfreq('d')

pasajeros_d = pasajeros_d['pas']
pasajeros_d.describe()

count       760.000000
mean     175952.678947
std       82497.043937
min        5582.000000
25%      106144.500000
50%      178343.000000
75%      249701.000000
max      319953.000000
Name: pas, dtype: float64

### Consumo diario

In [4]:
# Consumo
url = 'https://raw.githubusercontent.com/jiang-011/AST_G4/main/dataset/Consumo_TP.csv'
raw_csv_data = pd.read_csv(url, sep=',', decimal='.')
consumo_d = raw_csv_data.copy()
consumo_d = consumo_d[(consumo_d['Fecha'] >= '2021-10-01') & (consumo_d['Fecha'] <= '2023-10-30')]
consumo_d = consumo_d.loc[consumo_d.Consumo>0]
consumo_d.rename({'Fecha': 'fecha', 'Consumo': 'con'}, axis=1, inplace=True)
nuevas_observaciones = pd.DataFrame([{'fecha': '2022-05-17', 'con': 1},
                                     {'fecha': '2022-10-12', 'con': 1},
                                     {'fecha': '2022-10-13', 'con': 1},
                                     {'fecha': '2023-06-13', 'con': 1},
                                     {'fecha': '2023-06-22', 'con': 1}])
consumo_d = pd.concat([consumo_d, nuevas_observaciones], ignore_index=True)

# Ordenar el DataFrame por la columna de fechas
consumo_d = consumo_d.sort_values(by='fecha', ascending=True)

# Resetear el índice
consumo_d = consumo_d.reset_index(drop=True)

# Convertir la columna 'Fecha' a tipo datetime
consumo_d['fecha'] = pd.to_datetime(consumo_d['fecha'], dayfirst=True, format="%Y-%m-%d")
# Encontrar el índice correspondiente a '2022-05-17'
index_to_replace = consumo_d.index[consumo_d['fecha'] == '2022-05-17'][0]

# Calcular el promedio de una semana previa y una semana posterior
average_value_17 = (consumo_d.loc[index_to_replace - 7, 'con'] + consumo_d.loc[index_to_replace + 7, 'con']) / 2

# Reemplazar el valor en el DataFrame
consumo_d.loc[index_to_replace, 'con'] = average_value_17

# Encontrar el índice correspondiente a '2022-10-12'
index_to_replace = consumo_d.index[consumo_d['fecha'] == '2022-10-12'][0]

# Calcular el promedio de un día previo y un día posterior
average_value_12 = (consumo_d.loc[index_to_replace - 7, 'con'] + consumo_d.loc[index_to_replace + 7, 'con']) / 2

# Reemplazar el valor en el DataFrame
consumo_d.loc[index_to_replace, 'con'] = average_value_12

# Encontrar el índice correspondiente a '2022-10-13'
index_to_replace = consumo_d.index[consumo_d['fecha'] == '2022-10-13'][0]

# Calcular el promedio de un día previo y un día posterior
average_value_13 = (consumo_d.loc[index_to_replace - 7, 'con'] + consumo_d.loc[index_to_replace + 7, 'con']) / 2

# Reemplazar el valor en el DataFrame
consumo_d.loc[index_to_replace, 'con'] = average_value_13

# Encontrar el índice correspondiente a '2023-06-13'
index_to_replace = consumo_d.index[consumo_d['fecha'] == '2023-06-13'][0]

# Calcular el promedio de un día previo y un día posterior
average_value_6 = (consumo_d.loc[index_to_replace - 7, 'con'] + consumo_d.loc[index_to_replace + 7, 'con']) / 2

# Reemplazar el valor en el DataFrame
consumo_d.loc[index_to_replace, 'con'] = average_value_6

# Encontrar el índice correspondiente a '2023-06-22'
index_to_replace = consumo_d.index[consumo_d['fecha'] == '2023-06-22'][0]

# Calcular el promedio de un día previo y un día posterior
average_value_22 = (consumo_d.loc[index_to_replace - 7, 'con'] + consumo_d.loc[index_to_replace + 7, 'con']) / 2

# Reemplazar el valor en el DataFrame
consumo_d.loc[index_to_replace, 'con'] = average_value_22

# Formato diario
consumo_d.set_index('fecha', inplace=True)

consumo_d = consumo_d.asfreq('d')
consumo_d = consumo_d['con']
consumo_d.describe()

count    760.000000
mean      46.018249
std        3.446071
min       21.671538
25%       44.059896
50%       46.227106
75%       48.002559
max       55.802864
Name: con, dtype: float64

### KM recorridos dario

In [6]:
# KM
url='https://raw.githubusercontent.com/jiang-011/AST_G4/main/dataset/Kilometros_TP.csv'
raw_csv_data = pd.read_csv(url, sep=',', decimal='.')
km_d = raw_csv_data.copy()
km_d = km_d[(km_d['Fecha'] >= '2021-10-01') & (km_d['Fecha'] <= '2023-10-30')]

km_d.rename({'Fecha': 'fecha', 'KM': 'km'}, axis=1, inplace=True)
nuevas_observaciones = pd.DataFrame([{'fecha': '2023-06-22', 'km': 1}])
km_d = pd.concat([km_d, nuevas_observaciones], ignore_index=True)
km_d = km_d.sort_values(by=['fecha'], ascending=True)
km_d = km_d.reset_index(drop=True)
# Convertir la columna 'Fecha' a tipo datetime
km_d['fecha'] = pd.to_datetime(km_d['fecha'], dayfirst=True, format="%Y-%m-%d")
# Encontrar el índice correspondiente a '2022-05-17'
index_to_replace = km_d.index[km_d['fecha'] == '2022-05-17'][0]

# Calcular el promedio de una semana previa y una semana posterior
average_value_17 = (km_d.loc[index_to_replace - 7, 'km'] + km_d.loc[index_to_replace + 7, 'km']) / 2

# Reemplazar el valor en el DataFrame
km_d.loc[index_to_replace, 'km'] = average_value_17

# Encontrar el índice correspondiente a '2022-10-12'
index_to_replace = km_d.index[km_d['fecha'] == '2022-10-12'][0]

# Calcular el promedio de un día previo y un día posterior
average_value_12 = (km_d.loc[index_to_replace - 7, 'km'] + km_d.loc[index_to_replace + 7, 'km']) / 2

# Reemplazar el valor en el DataFrame
km_d.loc[index_to_replace, 'km'] = average_value_12

# Encontrar el índice correspondiente a '2022-10-13'
index_to_replace = km_d.index[km_d['fecha'] == '2022-10-13'][0]

# Calcular el promedio de un día previo y un día posterior
average_value_13 = (km_d.loc[index_to_replace - 7, 'km'] + km_d.loc[index_to_replace + 7, 'km']) / 2

# Reemplazar el valor en el DataFrame
km_d.loc[index_to_replace, 'km'] = average_value_13

# Encontrar el índice correspondiente a '2023-06-13'
index_to_replace = km_d.index[km_d['fecha'] == '2023-06-13'][0]

# Calcular el promedio de un día previo y un día posterior
average_value_6 = (km_d.loc[index_to_replace - 7, 'km'] + km_d.loc[index_to_replace + 7, 'km']) / 2

# Reemplazar el valor en el DataFrame
km_d.loc[index_to_replace, 'km'] = average_value_6

# Encontrar el índice correspondiente a '2023-06-22'
index_to_replace = km_d.index[km_d['fecha'] == '2023-06-22'][0]

# Calcular el promedio de un día previo y un día posterior
average_value_22 = (km_d.loc[index_to_replace - 7, 'km'] + km_d.loc[index_to_replace + 7, 'km']) / 2

# Reemplazar el valor en el DataFrame
km_d.loc[index_to_replace, 'km'] = average_value_22

# Formato diario
km_d.set_index('fecha', inplace=True)

km_d = km_d.asfreq('d')
km_d = km_d['km']
km_d.describe()

count       760.000000
mean      73977.713023
std       23424.039351
min       17554.422000
25%       52696.933750
50%       75397.900000
75%       97051.868000
max      105834.661000
Name: km, dtype: float64

### Temperatura

### Resumen

Se juntan todos los datos diarios disponibles que relacionan para predecir el consumo

In [8]:
# Dataframe de las tres series que se utilizarán para pycaret
df = pd.concat([consumo_d, km_d, pasajeros_d], axis=1)

Agregamos algunas variables rezagadas

In [9]:
df['pas_7'] = df['pas'].diff(7)
df['pas_5'] = df['pas'].diff(5)
df['con_7'] = df['con'].diff(7)
df['con_5'] = df['con'].diff(5)
df['km_7'] = df['km'].diff(7)
df['km_5'] = df['km'].diff(5)

In [11]:
# Formato diario
df.head()

,con,km,pas,pas_7,pas_5,con_7,con_5,km_7,km_5
fecha,,,,,,,,,
2021-10-01,45.201497,73177.190,163933,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-02,39.899719,49300.256,71862,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-03,39.705724,34024.972,41802,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-04,45.777167,73453.515,159940,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-05,46.147136,73722.635,170473,NaN,NaN,NaN,NaN,NaN,NaN
